In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

In [2]:
metrics = pd.read_csv('top_k_metrics_new.csv')
metrics.columns = [c.replace("('pred_prob', '",'prob_').replace("')",'') for c in metrics.columns]
metrics.columns = [c.replace("('pred', '",'pred_').replace("')",'') for c in metrics.columns]
metrics.columns = [c.replace("pred_prod_std",'ambiguity') for c in metrics.columns]
metrics.head()

,movie_fname,movieid,genre,prob_Action,prob_Comedy,prob_Drama,prob_Horror,prob_Romance,prob_Thriller,pred_Action,...,first_genre_in_top_2,first_two_genres_ranked_top,first_two_genres_in_top_2,second_genre_ranked_top,second_genre_in_top_2,D1,D2,D3,D4,A2
0,tt0000003,tt0000003,Comedy,0.059085,0.615839,0.596690,0.041991,0.234246,0.087320,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,tt0000005,tt0000005,Drama,0.024403,0.234024,0.911174,0.001686,0.226441,0.025258,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,tt0000014,tt0000014,Comedy,0.344066,0.408890,0.299501,0.384320,0.051391,0.215331,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tt0000131,tt0000131,Comedy|Horror,0.108410,0.269579,0.714259,0.057306,0.131395,0.100196,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,tt0000242,tt0000242,Horror|Comedy,0.042985,0.042881,0.894845,0.023537,0.588729,0.168944,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0


metrics['pred_genre'] = metrics.apply(lambda x: 'Action' if x['pred_Action'] == 1.0
                                                 else ('Comedy' if x['pred_Comedy'] == 1.0
                                                      else ('Drama' if x['pred_Drama'] == 1.0
                                                           else ( 'Horror' if x['pred_Horror'] == 1.0
                                                                else ('Romance' if x['pred_Romance'] == 1.0
                                                                     else ('Thriller' if x['pred_Thriller'] == 1.0
                                                                          else 'None'))))), axis = 1)

In [3]:
metrics.shape

(31568, 27)

In [4]:
imdb_meta = pd.read_excel('imdb_scraping_all_columns.xlsx')
imdb_meta.rename({'release_country (dummy = 1 if USA)':'release_country',
                  'runtime (not crucial)':'duration',
                 'num_raters (can convert to 1000 unit)':'num_raters',
                 'budget (can replace missing value with a small number e.g. $10,000) (can take log if DV is in log)':'budget',
                 'gross_US (can take log)':'gross_US',
                 'open_boxoffice_US (can take log; can do log (open box office/week 1 screen) and if so no week 1 screen as IV)':'open_boxoffice_US',
                 "release_date (if have time, convert to holiday dummy, e.g. 1 = week corresponding to  Valentine's Day, Memeorial Day, July 4th, Labor Day, Thanksgiving, Chrismas, New Year; if not time now can leave out for now)":'release_date',
                 'certificate (convert into dummies)':'certificate',
                 'genre (dummy coding; only focus on first genre and focus on the top 6 genres consistent with our poster analysis)':'actual_genre',
                 "other IVs may include studio (which can be scraped from imdb), competition (which can be calculated by the total number or total budget of all movies released in the same open weekend), star power (calculated as the Oscar win or past box office of the key cast and crew members), ad (which we don't have) - we can forgo these for now and do later ":'X_factor',
                 'metascore (divide by 10)':'meta_score',
                 'release_country (dummy = 1 if USA)':'release_country'}
                 , axis = 1, inplace = True)


# imdb_meta['release_country'] = imdb_meta['release_country'].fillna('USA').apply(lambda x: 1 if x == 'ÚSA' else 0)
imdb_meta.columns

Index(['Liu_ID', 'title', 'movieid', 'year', 'actual_genre', 'rating',
       'description', 'people', 'sales', 'certificate', 'duration', 'movieurl',
       'release_date', 'release_country', 'average_user_rating', 'num_raters',
       'meta_score', 'metascore_num_critics', 'num_user_reviews',
       'num_critics_reviews', 'popularity ',
       'People who liked this also liked', 'num_oscar_wins', 'oscar_wins',
       'oscar_nominations', 'num_other_wins', 'num_other_nominations',
       'story_line_or_plot_summary', 'synopsis_or_spoiler',
       'synopsis_overflow', 'tagline', 'country', 'language', 'budget',
       'open_boxoffice_US', 'gross_US', 'gross_world', 'week1_US_date',
       'week1_US_screens', 'week1_US_boxoffice', 'week2_US_date',
       'week2_US_screens', 'week2_US_boxoffice', 'week3_US_date',
       'week3_US_screens', 'week3_US_boxoffice', 'week4_US_date',
       'week4_US_screens', 'week4_US_boxoffice', 'week5_US_date',
       'week5_US_screens', 'week5_US_boxoffic

In [5]:
imdb_meta.shape

(12059, 67)

## Production House

In [6]:
com_df = pd.read_csv('movies_metadata.csv')
# com_df.dropna(subset=['production_companies'], inplace= True)
com_df['production_companies'] = com_df['production_companies'].fillna('na')
com_df.rename({'imdb_id':'movieid'}, inplace = True, axis = 1)
com_df.columns

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'movieid', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [7]:
import ast
lis = []
major_studios = ['Walt Disney Pictures', 'Warner Bros.', 'Universal Pictures', 'Columbia Pictures', 'Paramount Pictures','20th Century Fox']
for i in com_df['production_companies']:
#     print(i)
    if ('name' in i):
        dictionary = ast.literal_eval(i)
        if (dictionary[0]['name'] in major_studios):
            lis.append(1)
        else:
            lis.append(0)
    else:
        lis.append(0)

com_df['production_companies'] = lis

## Major Holidays

In [8]:
imdb_meta["release_date"] = imdb_meta["release_date"].apply(lambda x: pd.to_datetime(x))
imdb_meta["release_day"] = imdb_meta["release_date"].apply(lambda x: x.day)
imdb_meta["release_month"] = imdb_meta["release_date"].apply(lambda x: x.month)
imdb_meta["release_year"] = imdb_meta["release_date"].apply(lambda x: x.year)
imdb_meta["release_day_of_week"] = imdb_meta["release_date"].dt.dayofweek

imdb_meta['release_week'] = imdb_meta['release_date'].dt.week
imdb_meta['release_week'] = imdb_meta[['release_week', 'release_day_of_week']].apply(lambda x: x['release_week']+1
                                                                                     if x['release_day_of_week'] not in [0.0, 1.0]
                                                                                    else x['release_week'], axis = 1)
imdb_meta[['release_date', 'release_week', 'release_day_of_week']]

,release_date,release_week,release_day_of_week
0,2009-12-18,52.0,4.0
1,1997-12-19,52.0,4.0
2,2012-05-04,19.0,4.0
3,2015-06-12,25.0,4.0
4,2008-07-18,30.0,4.0
...,...,...,...
12054,NaT,NaN,NaN
12055,NaT,NaN,NaN
12056,NaT,NaN,NaN
12057,NaT,NaN,NaN


In [9]:
from datetime import date

import holidays

us_holidays = holidays.US()
us_holidays.values()


all_dates = pd.date_range(start='1/1/1960', end='1/08/2020').to_series().to_frame()
all_dates.columns = ['all_dates']
all_dates['holiday_type'] = all_dates.apply(lambda x: us_holidays.get(x['all_dates']), axis = 1)
all_dates['holiday'] = all_dates['holiday_type'].apply(lambda x: 1.0 if x is not None else 0.0)

all_dates["release_day"] = all_dates["all_dates"].apply(lambda x: x.day)
all_dates["release_month"] = all_dates["all_dates"].apply(lambda x: x.month)
all_dates["release_year"] = all_dates["all_dates"].apply(lambda x: x.year)
all_dates["release_day_of_week"] = all_dates["all_dates"].dt.dayofweek

all_dates['release_week'] = all_dates['all_dates'].dt.week
all_dates['release_week'] = all_dates[['release_week', 'release_day_of_week']].apply(lambda x: x['release_week']+1
                                                                                     if x['release_day_of_week'] not in [0.0, 1.0]
                                                                                    else x['release_week'], axis = 1)
holiday_weeks = all_dates.groupby(['release_week', 'release_year'], as_index=False)['holiday'].max()
holiday_weeks

,release_week,release_year,holiday
0,1,1960,0.0
1,1,1961,1.0
2,1,1962,1.0
3,1,1963,1.0
4,1,1965,0.0
...,...,...,...
3188,54,2005,1.0
3189,54,2009,0.0
3190,54,2010,1.0
3191,54,2015,0.0


In [10]:
imdb_meta.shape

imdb_meta = pd.merge(imdb_meta, holiday_weeks, on = ['release_week', 'release_year'])
imdb_meta.head()

,Liu_ID,title,movieid,year,actual_genre,rating,description,people,sales,certificate,...,week9_US_boxoffice,week10_US_date,week10_US_screens,week10_US_boxoffice,release_day,release_month,release_year,release_day_of_week,release_week,holiday
0,X1,Avatar,/title/tt0499549/,2009,Action | Adventure | Fantasy | Sci-Fi,7.9,A paraplegic marine dispatched to the moon Pan...,\n Dir: James Cameron\n With: Sam Worthi...,$761M,PG_13,...,34115501.0,2010 Feb 19,2581.0,21183640.0,18.0,12.0,2009.0,4.0,52.0,0.0
1,X3345,Did You Hear About the Morgans?,/title/tt1314228/,2009,Comedy | Drama | Romance,4.7,"In New York City, an estranged couple who witn...",\n Dir: Marc Lawrence\n With: Hugh Grant...,$29.6M,UNRATED,...,NaN,NaN,NaN,NaN,18.0,12.0,2009.0,4.0,52.0,0.0
2,X17122,Me and Orson Welles,/title/tt1175506/,2008,Drama,6.8,A teenager is cast in the Mercury Theatre prod...,\n Dir: Richard Linklater\n With: Zac Ef...,$1.19M,NOT_RATED,...,34118.0,2010 Jan 29,23.0,27015.0,18.0,12.0,2009.0,4.0,52.0,0.0
3,X28170,Father of My Children,/title/tt1356928/,2009,Drama,6.7,A film producer struggles with suicidal despair.,\n Dir: Mia Hansen-Løve\n With: Louis-Do...,$144K,R,...,NaN,NaN,NaN,NaN,16.0,12.0,2009.0,2.0,52.0,0.0
4,X11032,The End of Poverty?,/title/tt0903943/,2008 Documentary,Documentary,7.2,A phenomenal discourse on why poverty exists w...,\n Dir: Philippe Diaz\n With: Martin She...,$57.3K,PG_13,...,NaN,2010 Jan 29,2.0,773.0,16.0,12.0,2009.0,2.0,52.0,0.0


## Data Cleaning

In [11]:
%%time
imdb_meta['year'] = imdb_meta['year'].apply(lambda x: float(str(x)[:4]) if str(x)[:4] != "????" else np.nan).values.astype('float64')
imdb_meta['movieid'] = imdb_meta['movieid'].apply(lambda x: x.replace('/title/','').replace(' ','')[:-1])

imdb_meta['sales'] = imdb_meta['sales'].apply(lambda x: str(x).replace(" ",'')).apply(lambda x: x.replace('$',''))
imdb_meta['sales'] = imdb_meta['sales'].apply(lambda x: float(x[:-1])*1000000 if x[-1] == 'M'
                                                         else (float(x[:-1])*1000 if x[-1] == 'K'
                                                              else (np.nan if (x[-1] == '-' or x[-3:] == '#N/A')
                                                                   else float(x)))).values.astype('float64')

imdb_meta.head()

CPU times: user 33.4 ms, sys: 65 µs, total: 33.4 ms
Wall time: 32.1 ms


,Liu_ID,title,movieid,year,actual_genre,rating,description,people,sales,certificate,...,week9_US_boxoffice,week10_US_date,week10_US_screens,week10_US_boxoffice,release_day,release_month,release_year,release_day_of_week,release_week,holiday
0,X1,Avatar,tt0499549,2009.0,Action | Adventure | Fantasy | Sci-Fi,7.9,A paraplegic marine dispatched to the moon Pan...,\n Dir: James Cameron\n With: Sam Worthi...,761000000.0,PG_13,...,34115501.0,2010 Feb 19,2581.0,21183640.0,18.0,12.0,2009.0,4.0,52.0,0.0
1,X3345,Did You Hear About the Morgans?,tt1314228,2009.0,Comedy | Drama | Romance,4.7,"In New York City, an estranged couple who witn...",\n Dir: Marc Lawrence\n With: Hugh Grant...,29600000.0,UNRATED,...,NaN,NaN,NaN,NaN,18.0,12.0,2009.0,4.0,52.0,0.0
2,X17122,Me and Orson Welles,tt1175506,2008.0,Drama,6.8,A teenager is cast in the Mercury Theatre prod...,\n Dir: Richard Linklater\n With: Zac Ef...,1190000.0,NOT_RATED,...,34118.0,2010 Jan 29,23.0,27015.0,18.0,12.0,2009.0,4.0,52.0,0.0
3,X28170,Father of My Children,tt1356928,2009.0,Drama,6.7,A film producer struggles with suicidal despair.,\n Dir: Mia Hansen-Løve\n With: Louis-Do...,144000.0,R,...,NaN,NaN,NaN,NaN,16.0,12.0,2009.0,2.0,52.0,0.0
4,X11032,The End of Poverty?,tt0903943,2008.0,Documentary,7.2,A phenomenal discourse on why poverty exists w...,\n Dir: Philippe Diaz\n With: Martin She...,57300.0,PG_13,...,NaN,2010 Jan 29,2.0,773.0,16.0,12.0,2009.0,2.0,52.0,0.0


## Merging Data

In [12]:
multilabel_df = pd.merge(pd.merge(imdb_meta, metrics, on = 'movieid'), com_df[['movieid','production_companies']], on = 'movieid')
multilabel_df.head()

,Liu_ID,title,movieid,year,actual_genre,rating,description,people,sales,certificate,...,first_two_genres_ranked_top,first_two_genres_in_top_2,second_genre_ranked_top,second_genre_in_top_2,D1,D2,D3,D4,A2,production_companies
0,X1,Avatar,tt0499549,2009.0,Action | Adventure | Fantasy | Sci-Fi,7.9,A paraplegic marine dispatched to the moon Pan...,\n Dir: James Cameron\n With: Sam Worthi...,761000000.0,PG_13,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0
1,X3345,Did You Hear About the Morgans?,tt1314228,2009.0,Comedy | Drama | Romance,4.7,"In New York City, an estranged couple who witn...",\n Dir: Marc Lawrence\n With: Hugh Grant...,29600000.0,UNRATED,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,1
2,X17122,Me and Orson Welles,tt1175506,2008.0,Drama,6.8,A teenager is cast in the Mercury Theatre prod...,\n Dir: Richard Linklater\n With: Zac Ef...,1190000.0,NOT_RATED,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,X28170,Father of My Children,tt1356928,2009.0,Drama,6.7,A film producer struggles with suicidal despair.,\n Dir: Mia Hansen-Løve\n With: Louis-Do...,144000.0,R,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,X17232,Ricky,tt1189076,2009.0,Comedy | Drama | Fantasy | Mystery | Romance,5.8,Neddie Nerfhoffer has a dream. He wants to do ...,\n Dir: François Ozon\n With: Alexandra ...,2230.0,R,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0


In [13]:
multilabel_df.shape

(7455, 100)

In [14]:
multilabel_df.columns.to_list()

['Liu_ID',
 'title',
 'movieid',
 'year',
 'actual_genre',
 'rating',
 'description',
 'people',
 'sales',
 'certificate',
 'duration',
 'movieurl',
 'release_date',
 'release_country',
 'average_user_rating',
 'num_raters',
 'meta_score',
 'metascore_num_critics',
 'num_user_reviews',
 'num_critics_reviews',
 'popularity ',
 'People who liked this also liked',
 'num_oscar_wins',
 'oscar_wins',
 'oscar_nominations',
 'num_other_wins',
 'num_other_nominations',
 'story_line_or_plot_summary',
 'synopsis_or_spoiler',
 'synopsis_overflow',
 'tagline',
 'country',
 'language',
 'budget',
 'open_boxoffice_US',
 'gross_US',
 'gross_world',
 'week1_US_date',
 'week1_US_screens',
 'week1_US_boxoffice',
 'week2_US_date',
 'week2_US_screens',
 'week2_US_boxoffice',
 'week3_US_date',
 'week3_US_screens',
 'week3_US_boxoffice',
 'week4_US_date',
 'week4_US_screens',
 'week4_US_boxoffice',
 'week5_US_date',
 'week5_US_screens',
 'week5_US_boxoffice',
 'week6_US_date',
 'week6_US_screens',
 'week6_US

In [15]:
multilabel_df['release_country']

0          USA
1          USA
2          USA
3       France
4          USA
         ...  
7450       USA
7451       USA
7452        UK
7453       USA
7454    Mexico
Name: release_country, Length: 7455, dtype: object

In [16]:
multilabel_df['budget'] = multilabel_df['budget'].fillna(10000)
multilabel_df['budget'] = [x if x !=0 else 10000 for x in multilabel_df['budget']]
multilabel_df['budget'] = np.log(multilabel_df['budget'])

multilabel_df['certificate'] = multilabel_df['certificate'].fillna('NOT_RATED')

multilabel_df['release_country'] = multilabel_df['release_country'].fillna('USA')
multilabel_df['release_country'] = [1 if x == 'USA' else 0 for x in multilabel_df['release_country']]

multilabel_df['week1_US_screens'] = multilabel_df['week1_US_screens'].fillna(2)

multilabel_df['meta_score'] = multilabel_df['meta_score'] / 10

multilabel_df['open_boxoffice_US'] = np.log(multilabel_df['open_boxoffice_US'])
multilabel_df['open_boxoffice_US'].replace(-np.inf, np.nan, inplace=True)

multilabel_df['week1_US_boxoffice'] = np.log(multilabel_df['week1_US_boxoffice'])
multilabel_df['sales'] = np.log(multilabel_df['sales'])
multilabel_df['num_raters'] = np.log(multilabel_df['num_raters'])
#multilabel_df['opening_revenue_per_screen'] = multilabel_df['open_boxoffice_US']/multilabel_df['week1_US_screens']

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [17]:
multilabel_df['Action'] = [1 if 'Action' in x else 0 for x in multilabel_df['actual_genre']]
multilabel_df['Romance'] = [1 if 'Romance' in x else 0 for x in multilabel_df['actual_genre']]
multilabel_df['Drama'] = [1 if 'Drama' in x else 0 for x in multilabel_df['actual_genre']]
multilabel_df['Thriller'] = [1 if 'Thriller' in x else 0 for x in multilabel_df['actual_genre']]
multilabel_df['Horror'] = [1 if 'Horror' in x else 0 for x in multilabel_df['actual_genre']]
multilabel_df['Comedy'] = [1 if 'Comedy' in x else 0 for x in multilabel_df['actual_genre']]

In [18]:
multilabel_df[['Action','Romance','Drama','Thriller','Horror','Comedy','actual_genre']]

,Action,Romance,Drama,Thriller,Horror,Comedy,actual_genre
0,1,0,0,0,0,0,Action | Adventure | Fantasy | Sci-Fi
1,0,1,1,0,0,1,Comedy | Drama | Romance
2,0,0,1,0,0,0,Drama
3,0,0,1,0,0,0,Drama
4,0,1,1,0,0,1,Comedy | Drama | Fantasy | Mystery | Romance
...,...,...,...,...,...,...,...
7450,0,1,0,0,0,1,Comedy | Romance
7451,0,0,1,0,0,0,Drama
7452,0,0,0,0,1,0,Horror
7453,0,0,0,0,0,1,Comedy | Talk-Show


In [19]:
multilabel_df['certificate_above_R'] = multilabel_df['certificate'].apply(lambda x: 1 if x in ['R', 'NC_17'] else 0)
multilabel_df['certificate_below_R'] = multilabel_df['certificate'].apply(lambda x: 1 if x in ['G', 'PG_13', 'PG'] else 0)
multilabel_df['duration'] = multilabel_df['duration'].apply(lambda x: str(x)[:-6].replace(' ','')).apply(lambda x: np.nan if x=='' else int(x))
multilabel_df['duration'].fillna(multilabel_df['duration'].mean(), inplace=True)

## Prepping Data for Modeling

In [20]:
# Y_cols = ['sales','average_user_rating','open_boxoffice_US','gross_US','gross_world','week1_US_boxoffice','week1_US_screens']
Y_cols = ['sales','open_boxoffice_US','week1_US_boxoffice']
# X_cols = ['year', 'num_raters', 'budget', 'certificate', 'release_country', 'meta_score', 'holiday', 'production_companies'
#           'prob_Action', 'prob_Comedy', 'prob_Drama', 'prob_Horror', 
#           'prob_Romance', 'prob_Thriller', 'ambiguity', 'typicality', 
#           'first_genre_ranked_top', 'first_genre_in_top_2', 
#           'second_genre_ranked_top', 'second_genre_in_top_2'
#          ]


X_cols = ['average_user_rating', 'certificate_above_R', 'certificate_below_R', 'budget',
          'duration', 'meta_score', 'holiday', 'production_companies',
          'ambiguity','D1','D2','D3','D4','Action','Romance','Drama','Thriller','Horror','Comedy', 'num_oscar_wins', 'week1_US_screens']

In [21]:
df = multilabel_df[X_cols+Y_cols].copy()
df.isnull().sum()

average_user_rating        0
certificate_above_R        0
certificate_below_R        0
budget                     0
duration                   0
meta_score                 0
holiday                    0
production_companies       0
ambiguity                  0
D1                         0
D2                         0
D3                         0
D4                         0
Action                     0
Romance                    0
Drama                      0
Thriller                   0
Horror                     0
Comedy                     0
num_oscar_wins             0
week1_US_screens           0
sales                     55
open_boxoffice_US       1349
week1_US_boxoffice      1262
dtype: int64

In [22]:
X = df[X_cols].copy()

# Normalizing values of X
numerical_cols = ['average_user_rating','budget','meta_score','ambiguity', 'duration']

for col in numerical_cols:
    print(col)
    X[col]=(X[col]-X[col].mean())/X[col].std()

X = pd.get_dummies(X, drop_first=True, dummy_na=False)
Y = df[Y_cols]

average_user_rating
budget
meta_score
ambiguity
duration


In [23]:
X.corr().style.background_gradient()

,average_user_rating,certificate_above_R,certificate_below_R,budget,duration,meta_score,holiday,production_companies,ambiguity,D1,D2,D3,D4,Action,Romance,Drama,Thriller,Horror,Comedy,num_oscar_wins,week1_US_screens
average_user_rating,1.000000,0.059843,-0.078438,-0.061091,0.276626,0.412911,0.010023,-0.049300,0.012108,-0.086435,-0.100950,-0.103095,-0.103095,-0.107283,0.036315,0.328035,-0.085867,-0.168607,-0.154175,0.190451,-0.204682
certificate_above_R,0.059843,1.000000,-0.639065,0.007332,0.030754,0.007417,-0.009514,-0.024422,0.007181,-0.001272,-0.003878,-0.015237,-0.015237,0.009365,-0.028589,0.048145,0.102696,0.074857,-0.062565,0.017168,-0.059123
certificate_below_R,-0.078438,-0.639065,1.000000,0.204915,0.041894,-0.007819,0.015450,0.165248,-0.001185,0.033230,0.038478,0.037570,0.037570,0.072645,0.060517,-0.130371,-0.055848,-0.082126,0.110287,0.021098,0.309258
budget,-0.061091,0.007332,0.204915,1.000000,0.156838,0.142813,0.029007,0.230790,0.006417,0.079563,0.119056,0.081592,0.081592,0.284747,-0.039199,-0.215811,0.187371,0.075399,-0.022785,0.087947,0.556251
duration,0.276626,0.030754,0.041894,0.156838,1.000000,0.086745,0.042133,0.053368,0.010559,-0.019130,-0.009909,-0.034445,-0.034445,0.115141,0.070233,0.160422,0.034515,-0.117013,-0.186342,0.172482,0.074048
meta_score,0.412911,0.007417,-0.007819,0.142813,0.086745,1.000000,0.004798,-0.016967,0.000177,-0.031832,-0.031359,-0.037604,-0.037604,-0.047807,-0.003216,0.123202,-0.031347,-0.054730,-0.066816,0.143469,0.026955
holiday,0.010023,-0.009514,0.015450,0.029007,0.042133,0.004798,1.000000,0.013732,0.001281,0.001756,0.016174,-0.002880,-0.002880,0.027634,0.008436,0.015587,-0.003997,-0.016456,-0.010696,0.029595,0.023533
production_companies,-0.049300,-0.024422,0.165248,0.230790,0.053368,-0.016967,0.013732,1.000000,0.001726,0.050984,0.055112,0.061228,0.061228,0.111492,-0.016381,-0.144925,0.037717,-0.005931,0.058833,0.037933,0.335044
ambiguity,0.012108,0.007181,-0.001185,0.006417,0.010559,0.000177,0.001281,0.001726,1.000000,-0.043743,0.058808,-0.028807,-0.028807,0.000867,0.001913,-0.007587,0.002917,0.010558,0.005729,0.019110,0.005139
D1,-0.086435,-0.001272,0.033230,0.079563,-0.019130,-0.031832,0.001756,0.050984,-0.043743,1.000000,0.681705,0.718097,0.718097,0.231011,-0.032356,-0.196912,0.117286,0.075588,0.072088,0.006052,0.129550


## Modeling

### Open Box Office

In [24]:
target = 'open_boxoffice_US'

y = Y[Y[target].isnull()==False][target]
X_train = X[Y[target].isnull()==False]

In [25]:
X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     983.1
Date:                Fri, 29 May 2020   Prob (F-statistic):               0.00
Time:                        12:23:49   Log-Likelihood:                -11405.
No. Observations:                6106   AIC:                         2.285e+04
Df Residuals:                    6085   BIC:                         2.299e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    9.9105      0.086    115.339      0.000       9.742      10.079
average_user_rating     -0.0070      0.025     -0.281      0.779      -0.056       0.042
certificate_above_R      1.2140      0.054     22.451      0.000       1.108       1.320
certificate_below_R      1.4873      0.059     25.166      0.000       1.371       1.603
budget                   0.5062      0.026     19.570      0.000       0.456       0.557
duration                 0.1927      0.022      8.664      0.000       0.149       0.236
meta_score              -0.1969      0.025     -7.796      0.000      -0.246      -0.147
holiday                  0.0081      0.051      0.158      0.874      -0.092       0.108
production_companies     0.6184      0.065      9.448      0.000       0.490       0.747
ambiguity                0.0033      0.020      0.161      0.872      -0.036       0.043
D1                       0.0841      0.082      1.029      0.304      -0.076       0.244
D2                       0.0156      0.057      0.272      0.786      -0.097       0.128
D3                      -0.0603      0.035     -1.725      0.085      -0.129       0.008
D4                      -0.0603      0.035     -1.725      0.085      -0.129       0.008
Action                   0.2155      0.067      3.229      0.001       0.085       0.346
Romance                  0.1499      0.048      3.133      0.002       0.056       0.244
Drama                   -0.1982      0.060     -3.279      0.001      -0.317      -0.080
Thriller                 0.1962      0.057      3.463      0.001       0.085       0.307
Horror                   0.4353      0.082      5.281      0.000       0.274       0.597
Comedy                   0.2445      0.055      4.474      0.000       0.137       0.352
num_oscar_wins           0.2671      0.047      5.674      0.000       0.175       0.359
week1_US_screens         0.0016   2.35e-05     69.483      0.000       0.002       0.002
==============================================================================
Omnibus:                      157.844   Durbin-Watson:                   1.743
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              297.545
Skew:                          -0.190   Prob(JB):                     2.45e-65
Kurtosis:                       4.013   Cond. No.                     3.06e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.51e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [26]:
X_train

,const,average_user_rating,certificate_above_R,certificate_below_R,budget,duration,meta_score,holiday,production_companies,ambiguity,...,D3,D4,Action,Romance,Drama,Thriller,Horror,Comedy,num_oscar_wins,week1_US_screens
0,1.0,1.479517,0,1,1.548008,3.018362,1.518899,0.0,0,1.071706,...,1.0,1.0,1,0,0,0,0,0,3.0,3452.0
1,1.0,-1.554092,0,0,1.167814,-0.112555,-0.424245,0.0,1,0.851183,...,0.0,0.0,0,1,1,0,0,1,0.0,2718.0
2,1.0,0.468314,0,0,0.940509,0.471175,1.171909,0.0,0,-0.064368,...,0.0,0.0,0,0,1,0,0,0,0.0,4.0
3,1.0,0.367193,1,0,-1.172745,-0.377887,1.276006,0.0,0,-2.103978,...,0.0,0.0,0,0,1,0,0,0,0.0,1.0
4,1.0,-0.542889,1,0,-1.172745,-1.492281,0.477929,0.0,0,0.066145,...,0.0,0.0,0,1,1,0,0,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7440,1.0,1.681757,0,0,-1.172745,-0.059489,-1.361118,1.0,0,1.756458,...,0.0,0.0,0,0,1,0,0,1,0.0,2.0
7441,1.0,1.782877,1,0,-0.712297,-0.643219,-1.361118,1.0,0,0.628185,...,1.0,1.0,0,0,1,0,0,0,0.0,1.0
7442,1.0,-1.149611,1,0,0.555049,4.504221,-1.361118,0.0,0,0.676284,...,0.0,0.0,0,0,1,0,0,1,0.0,1.0
7443,1.0,1.782877,0,0,-1.172745,0.099711,1.310705,0.0,0,-0.005148,...,1.0,1.0,0,0,0,0,1,1,0.0,2.0


# Only D1 and other variables - Budget #

In [29]:
cols = [x for x in X_train.columns if x not in ['meta_score','D2', 'D3', 'week1_US_screens','D4','certificate_below_R', 'average_user_rating','Horror','Action','Romance', 'Drama','Thriller','Comedy','num_oscar_wins','duration']]

model = sm.OLS(y, X_train[cols]).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.441
Model:                            OLS   Adj. R-squared:                  0.440
Method:                 Least Squares   F-statistic:                     801.1
Date:                Fri, 29 May 2020   Prob (F-statistic):               0.00
Time:                        12:24:59   Log-Likelihood:                -14035.
No. Observations:                6106   AIC:                         2.808e+04
Df Residuals:                    6099   BIC:                         2.813e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   11.8752      0.065    183.292      0.000      11.748      12.002
certificate_above_R      0.0934      0.063      1.488      0.137      -0.030       0.216
budget                   1.7945      0.032     55.711      0.000       1.731       1.858
holiday                  0.0894      0.078      1.142      0.254      -0.064       0.243
production_companies     2.2080      0.097     22.872      0.000       2.019       2.397
ambiguity                0.0175      0.031      0.566      0.571      -0.043       0.078
D1                       0.5031      0.068      7.425      0.000       0.370       0.636
==============================================================================
Omnibus:                       42.781   Durbin-Watson:                   1.702
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.580
Skew:                          -0.207   Prob(JB):                     3.44e-10
Kurtosis:                       2.996   Cond. No.                         4.33
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Only D1 and other variables - Number of Screens #

In [30]:
cols = [x for x in X_train.columns if x not in ['meta_score','D2', 'D3', 'budget','D4','certificate_below_R', 'average_user_rating','Horror','Action','Romance', 'Drama','Thriller','Comedy','num_oscar_wins','duration']]

model = sm.OLS(y, X_train[cols]).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     2378.
Date:                Fri, 29 May 2020   Prob (F-statistic):               0.00
Time:                        12:25:17   Log-Likelihood:                -12128.
No. Observations:                6106   AIC:                         2.427e+04
Df Residuals:                    6099   BIC:                         2.432e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   10.3705      0.049    212.918      0.000      10.275      10.466
certificate_above_R      0.4129      0.046      8.969      0.000       0.323       0.503
holiday                  0.0440      0.057      0.768      0.442      -0.068       0.156
production_companies     0.8753      0.073     11.994      0.000       0.732       1.018
ambiguity                0.0134      0.023      0.594      0.553      -0.031       0.058
D1                       0.1395      0.050      2.800      0.005       0.042       0.237
week1_US_screens         0.0021   2.01e-05    105.295      0.000       0.002       0.002
==============================================================================
Omnibus:                      152.786   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              357.782
Skew:                          -0.053   Prob(JB):                     2.04e-78
Kurtosis:                       4.181   Cond. No.                     4.93e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.93e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Only D2 and other variables #

In [172]:
cols = [x for x in X_train.columns if x not in ['D1', 'D3', 'D4','certificate_below_R', 'average_user_rating','meta_score','num_oscar_wins','duration']]

model = sm.OLS(y, X_train[cols]).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     501.3
Date:                Thu, 21 May 2020   Prob (F-statistic):               0.00
Time:                        16:00:48   Log-Likelihood:                -13712.
No. Observations:                6106   AIC:                         2.745e+04
Df Residuals:                    6093   BIC:                         2.754e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   12.2439      0.095    128.402      0.000      12.057      12.431
certificate_above_R      0.1178      0.060      1.962      0.050     8.4e-05       0.235
budget                   1.5454      0.032     47.848      0.000       1.482       1.609
holiday                  0.1047      0.074      1.408      0.159      -0.041       0.250
production_companies     1.9769      0.092     21.426      0.000       1.796       2.158
ambiguity                0.0036      0.029      0.122      0.903      -0.054       0.061
D2                       0.0124      0.063      0.196      0.845      -0.112       0.137
Action                   1.1073      0.092     12.066      0.000       0.927       1.287
Romance                  0.4358      0.069      6.309      0.000       0.300       0.571
Drama                   -1.0224      0.074    -13.854      0.000      -1.167      -0.878
Thriller                 0.3924      0.082      4.769      0.000       0.231       0.554
Horror                   0.4886      0.118      4.158      0.000       0.258       0.719
Comedy                   0.3449      0.070      4.919      0.000       0.207       0.482
==============================================================================
Omnibus:                       85.695   Durbin-Watson:                   1.707
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.242
Skew:                          -0.270   Prob(JB):                     1.54e-20
Kurtosis:                       3.257   Cond. No.                         7.71
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Only D3 and other variables #

In [173]:
cols = [x for x in X_train.columns if x not in ['D1', 'D2', 'D4','Horror','Action','Romance', 'Drama','Thriller','Comedy','certificate_below_R', 'average_user_rating','meta_score','num_oscar_wins','duration']]

model = sm.OLS(y, X_train[cols]).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.439
Model:                            OLS   Adj. R-squared:                  0.438
Method:                 Least Squares   F-statistic:                     795.2
Date:                Thu, 21 May 2020   Prob (F-statistic):               0.00
Time:                        16:00:53   Log-Likelihood:                -14045.
No. Observations:                6106   AIC:                         2.810e+04
Df Residuals:                    6099   BIC:                         2.815e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   12.0229      0.056    214.181      0.000      11.913      12.133
certificate_above_R      0.0988      0.063      1.571      0.116      -0.024       0.222
budget                   1.7974      0.032     55.692      0.000       1.734       1.861
holiday                  0.0910      0.078      1.160      0.246      -0.063       0.245
production_companies     2.2059      0.097     22.799      0.000       2.016       2.396
ambiguity                0.0140      0.031      0.454      0.650      -0.047       0.075
D3                       0.3703      0.063      5.923      0.000       0.248       0.493
==============================================================================
Omnibus:                       42.739   Durbin-Watson:                   1.697
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.535
Skew:                          -0.207   Prob(JB):                     3.52e-10
Kurtosis:                       2.997   Cond. No.                         4.07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Drama Genre Regression with D1 #

In [174]:

#X_train =multilabel_df[[x for x in X_cols if x not in ['Action','Romance', 'Drama','Thriller','Comedy','certificate_below_R','D4','D2','D3','average_user_rating']]]
#week1_US_boxoffice
#open_boxoffice_US
#week1_US_screens
drama_df = multilabel_df[multilabel_df['Drama'] == 1]
drama_df = drama_df.dropna(subset=['week1_US_boxoffice'])


y = drama_df['week1_US_boxoffice']

X_train =drama_df[[x for x in X_cols if x not in ['Horror','Action','Romance', 'Drama','Thriller','Comedy','duration','meta_score','num_oscar_wins','certificate_below_R','D4','D2','D3','average_user_rating']]]

X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     week1_US_boxoffice   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     296.9
Date:                Thu, 21 May 2020   Prob (F-statistic):          7.24e-311
Time:                        16:01:04   Log-Likelihood:                -8364.4
No. Observations:                3684   AIC:                         1.674e+04
Df Residuals:                    3677   BIC:                         1.679e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    7.2625      0.243     29.861      0.000       6.786       7.739
certificate_above_R      0.2062      0.078      2.642      0.008       0.053       0.359
budget                   0.3515      0.011     32.461      0.000       0.330       0.373
holiday                  0.1967      0.098      2.012      0.044       0.005       0.388
production_companies     2.2606      0.126     17.916      0.000       2.013       2.508
ambiguity                0.2053      0.617      0.333      0.739      -1.005       1.416
D1                       0.3630      0.080      4.533      0.000       0.206       0.520
==============================================================================
Omnibus:                       32.502   Durbin-Watson:                   1.814
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.182
Skew:                           0.115   Prob(JB):                     3.40e-06
Kurtosis:                       2.666   Cond. No.                         231.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Drama Genre Regression with D2 #

In [175]:

#week1_US_boxoffice
#open_boxoffice_US
drama_df = multilabel_df[multilabel_df['Drama'] == 1]
drama_df = drama_df.dropna(subset=['week1_US_boxoffice'])


y = drama_df['week1_US_boxoffice']

X_train =drama_df[[x for x in X_cols if x not in ['Horror','Action','Romance', 'Drama','Thriller','Comedy','meta_score','duration','num_oscar_wins','certificate_below_R','D4','D1','D3','average_user_rating']]]

X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     week1_US_boxoffice   R-squared:                       0.328
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     298.6
Date:                Thu, 21 May 2020   Prob (F-statistic):          2.12e-312
Time:                        16:01:15   Log-Likelihood:                -8360.9
No. Observations:                3684   AIC:                         1.674e+04
Df Residuals:                    3677   BIC:                         1.678e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    7.4221      0.238     31.201      0.000       6.956       7.889
certificate_above_R      0.2043      0.078      2.621      0.009       0.051       0.357
budget                   0.3501      0.011     32.342      0.000       0.329       0.371
holiday                  0.1948      0.098      1.995      0.046       0.003       0.386
production_companies     2.2675      0.126     17.997      0.000       2.020       2.514
ambiguity               -0.1024      0.618     -0.166      0.868      -1.314       1.109
D2                       0.4112      0.078      5.261      0.000       0.258       0.564
==============================================================================
Omnibus:                       30.038   Durbin-Watson:                   1.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.732
Skew:                           0.114   Prob(JB):                     7.02e-06
Kurtosis:                       2.680   Cond. No.                         231.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Drama Genre Regression with D3 #

In [138]:

#week1_US_boxoffice
#open_boxoffice_US
drama_df = multilabel_df[multilabel_df['Drama'] == 1]
drama_df = drama_df.dropna(subset=['week1_US_boxoffice'])


y = drama_df['week1_US_boxoffice']

X_train =drama_df[[x for x in X_cols if x not in ['Horror','Action','Romance', 'Drama','Thriller','Comedy','meta_score','duration','num_oscar_wins','certificate_below_R','D4','D2','D1','average_user_rating']]]

X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     week1_US_boxoffice   R-squared:                       0.324
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     293.9
Date:                Thu, 21 May 2020   Prob (F-statistic):          2.76e-308
Time:                        15:57:07   Log-Likelihood:                -8370.4
No. Observations:                3684   AIC:                         1.675e+04
Df Residuals:                    3677   BIC:                         1.680e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    7.5903      0.240     31.592      0.000       7.119       8.061
certificate_above_R      0.2103      0.078      2.691      0.007       0.057       0.364
budget                   0.3523      0.011     32.487      0.000       0.331       0.374
holiday                  0.1923      0.098      1.963      0.050       0.000       0.384
production_companies     2.2889      0.126     18.122      0.000       2.041       2.537
ambiguity                0.1306      0.618      0.211      0.833      -1.081       1.342
D3                      -0.2320      0.079     -2.931      0.003      -0.387      -0.077
==============================================================================
Omnibus:                       35.724   Durbin-Watson:                   1.815
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.401
Skew:                           0.105   Prob(JB):                     1.85e-06
Kurtosis:                       2.643   Cond. No.                         231.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Interaction of D's with genre

In [31]:
#interaction_df = multiclass_df[X_cols+Y_cols+['genre']]
#interaction_df

In [32]:
interaction_df = pd.DataFrame()
interaction_df['D1_Action'] = multilabel_df['D1']*multilabel_df['Action']
interaction_df['D1_Romance'] = multilabel_df['D1']*multilabel_df['Romance']
interaction_df['D1_Drama'] = multilabel_df['D1']*multilabel_df['Drama']
interaction_df['D1_Thriller'] = multilabel_df['D1']*multilabel_df['Thriller']
interaction_df['D1_Horror'] = multilabel_df['D1']*multilabel_df['Horror']
interaction_df['D1_Comedy'] = multilabel_df['D1']*multilabel_df['Comedy']
interaction_df['open_boxoffice_US'] = multilabel_df['open_boxoffice_US']
interaction_df.head()

,D1_Action,D1_Romance,D1_Drama,D1_Thriller,D1_Horror,D1_Comedy,open_boxoffice_US
0,1.0,0.0,0.0,0.0,0.0,0.0,18.159647
1,0.0,1.0,1.0,0.0,0.0,1.0,15.705088
2,0.0,0.0,0.0,0.0,0.0,0.0,11.060966
3,0.0,0.0,0.0,0.0,0.0,0.0,9.250906
4,0.0,0.0,0.0,0.0,0.0,0.0,7.393878


In [33]:
model = smf.ols(formula = 'open_boxoffice_US ~ D1*Action + D1*Romance + D1*Drama + D1*Thriller + D1*Horror + D1*Comedy', data = multilabel_df).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     132.0
Date:                Sun, 17 May 2020   Prob (F-statistic):          5.40e-316
Time:                        14:51:21   Log-Likelihood:                -15051.
No. Observations:                6106   AIC:                         3.013e+04
Df Residuals:                    6092   BIC:                         3.022e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      12.6593      0.222     56.969      0.000      12.224      13.095
D1              0.1684      0.256      0.657      0.511      -0.334       0.670
Action          2.0342      0.307      6.623      0.000       1.432       2.636
D1:Action       0.1311      0.330      0.397      0.691      -0.515       0.778
Romance         0.8348      0.151      5.545      0.000       0.540       1.130
D1:Romance     -0.3269      0.183     -1.782      0.075      -0.686       0.033
Drama          -1.7727      0.198     -8.973      0.000      -2.160      -1.385
D1:Drama        0.0219      0.222      0.099      0.921      -0.414       0.458
Thriller        0.8876      0.204      4.344      0.000       0.487       1.288
D1:Thriller    -0.2227      0.236     -0.944      0.345      -0.685       0.240
Horror          1.0485      0.316      3.315      0.001       0.429       1.669
D1:Horror      -0.5646      0.357     -1.584      0.113      -1.264       0.134
Comedy          0.4390      0.165      2.659      0.008       0.115       0.763
D1:Comedy      -0.0945      0.196     -0.483      0.629      -0.478       0.289
==============================================================================
Omnibus:                      131.123   Durbin-Watson:                   1.519
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               96.729
Skew:                          -0.212   Prob(JB):                     9.90e-22
Kurtosis:                       2.552   Cond. No.                         26.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
model = smf.ols(formula = 'open_boxoffice_US ~ D1_Action + D1_Romance + D1_Drama + D1_Thriller+ D1_Horror + D1_Comedy', data = interaction_df).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     211.0
Date:                Sun, 17 May 2020   Prob (F-statistic):          2.35e-245
Time:                        14:51:21   Log-Likelihood:                -15233.
No. Observations:                6106   AIC:                         3.048e+04
Df Residuals:                    6099   BIC:                         3.053e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      12.1999      0.061    200.651      0.000      12.081      12.319
D1_Action       2.4585      0.111     22.108      0.000       2.240       2.676
D1_Romance      0.6191      0.106      5.846      0.000       0.411       0.827
D1_Drama       -1.3921      0.082    -17.034      0.000      -1.552      -1.232
D1_Thriller     0.8204      0.118      6.962      0.000       0.589       1.051
D1_Horror       0.8613      0.154      5.576      0.000       0.558       1.164
D1_Comedy       0.7038      0.085      8.287      0.000       0.537       0.870
==============================================================================
Omnibus:                      324.059   Durbin-Watson:                   1.504
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              143.293
Skew:                          -0.151   Prob(JB):                     7.66e-32
Kurtosis:                       2.313   Cond. No.                         5.47
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
model = smf.ols(formula = 'open_boxoffice_US ~ D2*Action + D2*Romance + D2*Drama + D2*Thriller + D2*Horror + D2*Comedy', data = multilabel_df).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     132.2
Date:                Sun, 17 May 2020   Prob (F-statistic):          1.87e-316
Time:                        14:52:35   Log-Likelihood:                -15050.
No. Observations:                6106   AIC:                         3.013e+04
Df Residuals:                    6092   BIC:                         3.022e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      12.5663      0.168     74.726      0.000      12.237      12.896
D2              0.4175      0.225      1.860      0.063      -0.023       0.858
Action          2.0838      0.222      9.369      0.000       1.648       2.520
D2:Action       0.0175      0.259      0.068      0.946      -0.491       0.526
Romance         0.7431      0.118      6.310      0.000       0.512       0.974
D2:Romance     -0.2748      0.172     -1.596      0.111      -0.612       0.063
Drama          -1.6939      0.147    -11.500      0.000      -1.983      -1.405
D2:Drama       -0.0837      0.188     -0.446      0.655      -0.451       0.284
Thriller        0.8556      0.163      5.238      0.000       0.535       1.176
D2:Thriller    -0.2396      0.209     -1.144      0.252      -0.650       0.171
Horror          0.8771      0.247      3.547      0.000       0.392       1.362
D2:Horror      -0.4760      0.308     -1.544      0.123      -1.081       0.129
Comedy          0.5427      0.129      4.220      0.000       0.291       0.795
D2:Comedy      -0.3480      0.178     -1.952      0.051      -0.698       0.001
==============================================================================
Omnibus:                      130.071   Durbin-Watson:                   1.519
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               96.431
Skew:                          -0.212   Prob(JB):                     1.15e-21
Kurtosis:                       2.554   Cond. No.                         19.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
model = smf.ols(formula = 'open_boxoffice_US ~ D3*Action + D3*Romance + D3*Drama + D3*Thriller + D3*Horror + D3*Comedy', data = multilabel_df).fit()
# predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      open_boxoffice_US   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     133.6
Date:                Sun, 17 May 2020   Prob (F-statistic):          1.75e-319
Time:                        14:53:43   Log-Likelihood:                -15043.
No. Observations:                6106   AIC:                         3.011e+04
Df Residuals:                    6092   BIC:                         3.021e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      12.5308      0.188     66.603      0.000      12.162      12.900
D3              0.4658      0.237      1.964      0.050       0.001       0.931
Action          2.0061      0.176     11.366      0.000       1.660       2.352
D3:Action       0.1747      0.224      0.779      0.436      -0.265       0.614
Romance         0.9181      0.121      7.575      0.000       0.680       1.156
D3:Romance     -0.6041      0.172     -3.519      0.000      -0.941      -0.268
Drama          -1.5906      0.158    -10.058      0.000      -1.901      -1.281
D3:Drama       -0.3701      0.204     -1.814      0.070      -0.770       0.030
Thriller        0.9719      0.166      5.853      0.000       0.646       1.297
D3:Thriller    -0.4107      0.210     -1.952      0.051      -0.823       0.002
Horror          0.7786      0.273      2.847      0.004       0.242       1.315
D3:Horror      -0.3155      0.325     -0.972      0.331      -0.952       0.321
Comedy          0.4347      0.123      3.523      0.000       0.193       0.677
D3:Comedy      -0.1991      0.179     -1.110      0.267      -0.551       0.153
==============================================================================
Omnibus:                      128.750   Durbin-Watson:                   1.518
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               95.303
Skew:                          -0.210   Prob(JB):                     2.02e-21
Kurtosis:                       2.555   Cond. No.                         19.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### meta_score

In [188]:
y = multilabel_df['meta_score']
#budget
#production_companies
#certificate_above_R
#duration
X_train =multilabel_df[[x for x in X_cols if x not in ['num_oscar_wins','meta_score','holiday','D2','Horror','Action','Romance', 'Drama','Thriller','Comedy','D3' ,'certificate_below_R','average_user_rating', 'D4']]]
X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             meta_score   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     37.04
Date:                Thu, 21 May 2020   Prob (F-statistic):           1.67e-44
Time:                        16:08:25   Log-Likelihood:                -18359.
No. Observations:                7455   AIC:                         3.673e+04
Df Residuals:                    7448   BIC:                         3.678e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.5804      0.266      5.951      0.000       1.060       2.101
certificate_above_R      0.0176      0.067      0.263      0.792      -0.114       0.149
budget                   0.1151      0.009     12.420      0.000       0.097       0.133
duration                 0.0100      0.002      5.662      0.000       0.007       0.013
production_companies    -0.4398      0.098     -4.466      0.000      -0.633      -0.247
ambiguity               -0.1423      0.519     -0.274      0.784      -1.159       0.875
D1                      -0.2501      0.072     -3.467      0.001      -0.392      -0.109
==============================================================================
Omnibus:                    10730.566   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              507.179
Skew:                          -0.137   Prob(JB):                    7.37e-111
Kurtosis:                       1.752   Cond. No.                     1.79e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.79e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [189]:
y = multilabel_df['meta_score']
#budget
#production_companies
#certificate_above_R
#duration
X_train =multilabel_df[[x for x in X_cols if x not in ['num_oscar_wins','meta_score','holiday','D1','Horror','Action','Romance', 'Drama','Thriller','Comedy','D3' ,'certificate_below_R','average_user_rating', 'D4']]]
X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             meta_score   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     37.68
Date:                Thu, 21 May 2020   Prob (F-statistic):           2.68e-45
Time:                        16:09:18   Log-Likelihood:                -18357.
No. Observations:                7455   AIC:                         3.673e+04
Df Residuals:                    7448   BIC:                         3.678e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.4601      0.261      5.604      0.000       0.949       1.971
certificate_above_R      0.0166      0.067      0.248      0.804      -0.114       0.148
budget                   0.1168      0.009     12.569      0.000       0.099       0.135
duration                 0.0100      0.002      5.657      0.000       0.007       0.013
production_companies    -0.4401      0.098     -4.471      0.000      -0.633      -0.247
ambiguity                0.0582      0.519      0.112      0.911      -0.959       1.076
D2                      -0.2639      0.066     -3.969      0.000      -0.394      -0.134
==============================================================================
Omnibus:                    10611.872   Durbin-Watson:                   1.602
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              507.067
Skew:                          -0.139   Prob(JB):                    7.80e-111
Kurtosis:                       1.753   Cond. No.                     1.78e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [192]:
y = multilabel_df['meta_score']
#budget
#production_companies
#certificate_above_R
#duration
X_train =multilabel_df[[x for x in X_cols if x not in ['num_oscar_wins','meta_score','holiday','D1','D2','Horror','Action','Romance', 'Drama','Thriller','Comedy' ,'certificate_below_R','average_user_rating', 'D4']]]
X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             meta_score   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     37.53
Date:                Thu, 21 May 2020   Prob (F-statistic):           4.06e-45
Time:                        16:09:45   Log-Likelihood:                -18357.
No. Observations:                7455   AIC:                         3.673e+04
Df Residuals:                    7448   BIC:                         3.678e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.5487      0.263      5.887      0.000       1.033       2.064
certificate_above_R      0.0142      0.067      0.212      0.832      -0.117       0.145
budget                   0.1154      0.009     12.458      0.000       0.097       0.134
duration                 0.0099      0.002      5.586      0.000       0.006       0.013
production_companies    -0.4347      0.098     -4.413      0.000      -0.628      -0.242
ambiguity               -0.1208      0.518     -0.233      0.816      -1.137       0.895
D3                      -0.2566      0.066     -3.861      0.000      -0.387      -0.126
==============================================================================
Omnibus:                    10257.395   Durbin-Watson:                   1.602
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              504.898
Skew:                          -0.139   Prob(JB):                    2.31e-110
Kurtosis:                       1.756   Cond. No.                     1.78e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
y = multilabel_df['meta_score']
X_train = multilabel_df[[x for x in X_cols if x not in ['meta_score', 'average_user_rating', 'holiday', 'certificate_below_R']]]
X_train = sm.add_constant(X_train)

model = sm.OLS(y, X_train).fit()

# Print out the statistics
model.summary()